In [24]:
from timeit import timeit

## Cache
You can cache the results of a function or method depending on the input.

The `@cache` decorator caches every result.
The `@lru_cache` decorator caches the _L_east _R_ecently _U_sed and takes a size parameter,
indicating how many results are cached at once before deleting the least used return.

In [31]:
from functools import cache, lru_cache

def fib(n):
    if n <= 1:
        return n
    return fib(n-1) + fib(n-2)

@cache
def cache_fib(n):
    return fib(n)

@lru_cache(5)
def lru_cache_fib(n):
    return fib(n)

print(timeit(lambda: fib(25), number=50))
print(timeit(lambda: cache_fib(25), number=50))
print(timeit(lambda: lru_cache_fib(25), number=50))


1.9948670320009114
0.0334765570005402
0.03308480700070504


Another great caching decorator is `functools.cached_property`. It combines the benefits of `@property` and `@cache` into one single decorator. Although you have to be careful, since the cached property does not look at the state of the class, so you have to clear the cache manually if you know something inside the object changed.

In [32]:
from functools import cached_property
import time
class A:
    def __init__(self):
        self._foo = 1

    @cached_property
    def foo(self):
        time.sleep(1)
        return self._foo

a = A()
print(timeit(lambda: a.foo, number=3000))

a._foo = 2  # evil
print(timeit(lambda: a.foo, number=3000))

del a.foo
print(timeit(lambda: a.foo, number=3000))

1.0020449720032047
0.000667869986500591
1.0029271800012793


## Single dispatch

With `functools.singledispatch` you can actually have generic functions, a.k.a. function overloading!
Just define a function and decorate it with `@singledispatch`.
Define overloading functions by adding the `register` decorator.


In [29]:
from functools import singledispatch
@singledispatch
def fun(arg, verbose=False):
    if verbose:
        print("Let me just say,", end=" ")
    print(arg)

@fun.register
def _(arg: int, verbose=False):
    if verbose:
        print("Strength in numbers, eh?", end=" ")
    print(arg)

@fun.register
def _(arg: list, verbose=False):
    if verbose:
        print("Enumerate this:")
    for i, elem in enumerate(arg):
        print(i, elem)

fun(69, True)
fun(['foo', 'bar'], True)


Strength in numbers, eh? 49
Enumerate this:
0 foo
1 bar


Note that this best works with type overloading.
Overloading with a different number of arguments only works when not interfering with any other overload types.
Also my editor was not happy about this, so it's best to avoid it.

In [2]:
from functools import singledispatch

@singledispatch
def fun(arg1, arg2):
    print(arg1, arg2)

@fun.register
def _(arg1: int, arg2: int):
    print(arg1 + arg2)

@fun.register
def _(arg1: float, arg2: float, arg3: float):
    print(arg1, arg2, arg3)

fun('lol', 'lmao')
fun(1, 2)
fun(1., 2., 3.)

lol lmao
3
1.0 2.0 3.0


## Reduce
Turn an iterable into a single value by some condition

In [32]:
from functools import reduce

# adding all values
my_sum = reduce(lambda x, y: x + y, [1, 2, 3, 4])
print(my_sum)

# find the biggest number in a list
reduce(lambda x, y: y if y > x else x, [0, 3, 2, 5, 4, 5, 6, 2])


10


6